In [10]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = "/Users/srdeo/OneDrive - Copart, Inc/secrets/cprtpr-datastewards-sp1-614d7e297848 (1).json"
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### For buyers placing bids:

In [38]:
## For active buyers

query1 = f"""
with filters as
(
select f.mbr_lic_type, f.mbr_state, b.lot_nbr, b.buyer_nbr, f.mbr_email, b.bid_amt, l.inv_dt, l.lot_year, l.lot_make_cd, l.grp_model, l.acv, l.repair_cost
from `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
join `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
on b.lot_nbr = l.lot_nbr
join `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
on f.mbr_nbr = b.buyer_nbr
where l.inv_dt between '2025-07-13' and '2025-10-13'
and lot_type_cd = 'V' and yard_country_cd = 'USA'
and mbr_country = 'USA' AND mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
  AND mbr_site_status_cd = 'A'
AND mbr_mbrshp_status_cd = 'A' AND mbr_lang_pref = 'en'
AND mbr_status = 'A'
),
lots_bid_counts as (
 select lot_nbr, count(distinct buyer_nbr) as total_unique_buyers_on_that_lot
    from filters
    group by 1
 ),
 buyers_bid_counts as (
 select buyer_nbr, count(distinct lot_nbr) as total_unique_lots_bid_by_buyers
 from filters
 group by 1
 )
 select f.mbr_lic_type, f.mbr_state, f.lot_nbr, f.buyer_nbr, f.mbr_email, max(f.bid_amt) as max_bid, f.inv_dt, f.lot_year, f.lot_make_cd, f.grp_model, f.acv, f.repair_cost, lb.total_unique_buyers_on_that_lot, bb.total_unique_lots_bid_by_buyers
 from filters f
 left join lots_bid_counts as lb on f.lot_nbr = lb.lot_nbr
 left join buyers_bid_counts as bb on f.buyer_nbr = bb.buyer_nbr
 GROUP BY
    f.mbr_lic_type, f.mbr_state, f.lot_nbr, f.buyer_nbr, f.mbr_email, f.inv_dt, f.lot_year, f.lot_make_cd, f.grp_model,
    f.acv, f.repair_cost, lb.total_unique_buyers_on_that_lot, bb.total_unique_lots_bid_by_buyers
"""

data1 = client.query(query1).to_dataframe()

In [39]:
active_buyers = data1.copy()

In [40]:
active_buyers

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
0,Consumer,TN,60112235,307475,307475cprt_dmmy_MASTERBDR_774468_468903@copart...,525.0,2025-08-26,2013,BUIC,VERANO,6225.00,6328.59,4,53416
1,Consumer,PA,81847425,649757,Narciso1967jb@gmail.com,150.0,2025-10-07,2007,HYUN,VERACRUZ,4525.00,0.00,2,28
2,Dealer,NJ,65945235,889115,jimotorsllc2020@gmail.com,3000.0,2025-09-30,2022,ACUR,TLX,32572.56,26578.75,15,105
3,Dealer,OH,69207575,18268,gmcautosale@gmail.com,2500.0,2025-09-23,2016,NISS,MAXIMA,9756.00,5215.90,5,244
4,General Business,CA,69213575,211748,ghafar_mushtaq@yahoo.com,3950.0,2025-08-28,2018,VOLK,GTI,22351.00,13044.91,3,506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3848018,Dealer,FL,62964545,544244,10autollc@gmail.com,9600.0,2025-08-05,2025,JEP,WRANGLER,42026.00,16530.25,12,242
3848019,Consumer,KY,60941985,86729,86729cprt_dmmy_MASTERBDR_75225_344013@copart.com,3000.0,2025-08-19,2010,JEP,WRANGLER,16826.00,0.00,16,51
3848020,Consumer,FL,68889025,645763,251973-MSTRBDR-0bf4ba67-6ac0-4be5-9c08-fbc056e...,3500.0,2025-08-21,2014,JEP,WRANGLER,18000.00,0.00,5,7
3848021,Consumer,OR,67040905,622926,622926cprt_dmmy_BROKERBDR_341095_557710@copart...,2000.0,2025-08-26,1971,JEP,WRANGLER,15895.00,0.00,7,2


In [41]:
active_buyers['buyer_nbr'].nunique()

80888

In [42]:
active_buyers.to_csv('data/raw/active_bidders.csv',index=False)

In [59]:
active_buyers[active_buyers['buyer_nbr']==156698]

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
88495,Consumer,IN,55850725,156698,benwmichaels36@icloud.com,1300.0,2025-08-14,2018,LAND,RANGEROVER,14625.00,0.0,36,8
126571,Consumer,IN,57502915,156698,benwmichaels36@icloud.com,125.0,2025-07-23,2013,BMW,5 SERIES,11025.00,11025.0,6,8
834400,Consumer,IN,59674335,156698,benwmichaels36@icloud.com,950.0,2025-07-18,2016,BMW,X3,4485.00,0.0,13,8
1271525,Consumer,IN,56826135,156698,benwmichaels36@icloud.com,1150.0,2025-07-16,2011,BMW,5 SERIES,2558.00,4897.6,7,8
1289568,Consumer,IN,43960615,156698,benwmichaels36@icloud.com,250.0,2025-08-11,2018,FORD,F250,38885.00,37393.0,12,8
1290007,Consumer,IN,61119935,156698,benwmichaels36@icloud.com,3000.0,2025-08-28,2017,BMW,X3,0.00,0.0,21,8
1336302,Consumer,IN,57714695,156698,benwmichaels36@icloud.com,500.0,2025-08-01,2012,SUBA,IMPREZA,9899.00,0.0,17,8
1353586,Consumer,IN,58225065,156698,benwmichaels36@icloud.com,850.0,2025-08-05,2024,BMW,M3,59627.55,0.0,46,8


#### For buyers placing no bids

In [60]:
## Buyers with no bids

query2 = f"""
SELECT f.mbr_lic_type, f.mbr_state, f.mbr_nbr, f.mbr_email
FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
left join `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
ON f.mbr_nbr = b.buyer_nbr
and b.auc_dt between '2025-07-13' and '2025-10-13'
WHERE mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
  AND mbr_status = 'A'
  AND mbr_site_status_cd = 'A'
 and bid_id is null
 and mbr_country = 'USA'
"""

data2 = client.query(query2).to_dataframe()

In [61]:
data2

,mbr_lic_type,mbr_state,mbr_nbr,mbr_email
0,Export,VI,840303,v_benjamin05@hotmail.com
1,Consumer,VT,471987,realestate7891@yahoo.com
2,Consumer,WV,546400,rpm_1911@hotmail.com
3,Consumer,AK,103653,miller68000@gmail.com
4,Consumer,AK,446562,jacobstulic@yahoo.com
...,...,...,...,...
145384,Consumer,WI,250615,felipeu536@gmail.com
145385,Dealer,WI,471791,takalafomond@gmail.com
145386,Consumer,WI,923382,kavanderaa@gmail.com
145387,Consumer,WI,651771,jacksellshouses0@gmail.com


In [62]:
data2['mbr_nbr'].nunique()

145389

In [46]:
data2.to_csv('data/raw/not_active_bidders.csv', index=False)

In [63]:
data2[data2['mbr_nbr']==156698]

,mbr_lic_type,mbr_state,mbr_nbr,mbr_email
53242,Consumer,IN,156698,benwmichaels36@icloud.com


### Future Lots

In [47]:
## Future lots

query5 = """
SELECT
  lot_nbr, lot_year, lot_make_cd, grp_model, damage_type_desc, repair_cost, acv, auc_dt, COALESCE(proquote_amt, proquote_amt_new) AS proquote_amt
FROM `cprtpr-dataplatform-sp1.usviews.v_us_lot_fact` l
WHERE l.auc_dt BETWEEN DATE_ADD(CURRENT_DATE(), INTERVAL 1 DAY) AND DATE_ADD(CURRENT_DATE(), INTERVAL 7 DAY)
  and loss_type_cd not in ('B','K','F')
    and damage_type_cd not in ('BN','BE','BI','BC')
        and lot_type_cd = 'V' and yard_country_cd = 'USA'
  and offsite_flag = 'N'
"""

data5 = client.query(query5).to_dataframe()

In [48]:
data5.to_csv('data/raw/future_lots.csv', index=False)

In [49]:
data5

,lot_nbr,lot_year,lot_make_cd,grp_model,damage_type_desc,repair_cost,acv,auc_dt,proquote_amt
0,71239515,2017,VOLK,PASSAT,FRONT END,9251.75,9841.73,2025-10-14,2544.05
1,81929665,2013,HYUN,VELOSTER,FRONT END,0.00,4250.00,2025-10-14,1225.38
2,67987075,2016,SUBA,CROSSTREK,FRAME DAMAGE,6898.95,7043.00,2025-10-15,1722.39
3,81834465,2012,CADI,CTS,FRONT END,0.00,608.00,2025-10-16,503.58
4,65324015,2014,CHEV,SONIC,FRONT END,3261.53,3927.00,2025-10-15,679.09
...,...,...,...,...,...,...,...,...,...
44734,84478595,2017,JEP,WRANGLER,REAR END,0.00,0.00,2025-10-14,10884.31
44735,69480705,2019,JEP,WRANGLER,REJECTED REPAIR,15088.55,19247.00,2025-10-14,9896.34
44736,67549015,1998,JEP,WRANGLER,SIDE,10316.37,8457.48,2025-10-17,1334.83
44737,81210975,2019,JEP,WRANGLER,SIDE,28172.36,33560.00,2025-10-17,13111.74


## Popular Lots

In [50]:
query6 = """
WITH cte AS (
  SELECT
    l.buyer_type, f.mbr_state, l.lot_make_cd, l.grp_model,COUNT(*) AS cnt,
    APPROX_QUANTILES(l.acv, 2)[OFFSET(1)] AS median_acv, APPROX_QUANTILES(l.repair_cost, 2)[OFFSET(1)] AS median_repair_cost
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
  LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON f.mbr_nbr = l.buyer_nbr
  WHERE f.mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
    AND f.mbr_status = 'A' AND f.mbr_country = 'USA' AND f.mbr_site_status_cd = 'A'
    AND l.inv_dt BETWEEN '2025-07-13' and '2025-10-13'
    AND l.lot_year >= 2017
    AND l.lot_type_cd = 'V' AND l.yard_country_cd = 'USA'
  GROUP BY l.buyer_type, f.mbr_state, l.lot_make_cd, l.grp_model
),

deduped AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY buyer_type, mbr_state, lot_make_cd, grp_model
           ORDER BY cnt DESC
         ) AS model_rank
  FROM cte
),

ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY buyer_type, mbr_state
           ORDER BY cnt DESC
         ) AS rank
  FROM deduped
  WHERE model_rank = 1
)

SELECT *
FROM ranked
WHERE rank <= 6
"""

data6 = client.query(query6).to_dataframe()


In [51]:
data6

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_repair_cost,model_rank,rank
0,None,TX,NISS,SENTRA,4,12869.00,0.00,1,1
1,None,TX,JEP,WRANGLER,2,26515.11,11974.95,1,2
2,None,TX,BMW,4 SERIES,1,26481.00,25033.87,1,3
3,None,TX,HOND,CRV,1,38924.00,0.00,1,4
4,None,TX,FORD,FOCUS,1,7519.00,6124.27,1,5
...,...,...,...,...,...,...,...,...,...
1313,General Business,AZ,JEP,WRANGLER,1,22683.00,33296.82,1,2
1314,General Business,AZ,RAM,1500,1,23600.00,0.00,1,3
1315,General Business,AZ,MERZ,SPRINTER,1,91985.00,42286.00,1,4
1316,General Business,AZ,FORD,F150,1,0.00,0.00,1,5


In [52]:
data6[(data6['buyer_type']=='Consumer') & (data6['mbr_state']=='AL')]

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_repair_cost,model_rank,rank
975,Consumer,AL,HOND,CIVIC,3,11332.31,11332.31,1,1
976,Consumer,AL,CHEV,SILVERADO,3,51701.00,31738.45,1,2
977,Consumer,AL,CHEV,CAMARO,2,16660.00,2445.03,1,3
978,Consumer,AL,RAM,2500,2,37087.00,10388.99,1,4
979,Consumer,AL,FORD,EXPLORER,2,16645.94,0.00,1,5
980,Consumer,AL,NISS,ALTIMA,2,8824.00,7628.77,1,6


In [53]:
data6['buyer_type'].value_counts()

buyer_type
Dealer                         306
Consumer                       294
Dismantler                     269
General Business               199
Export                         160
Automotive Related Business     48
Name: count, dtype: int64

In [54]:
data6.to_csv('data/raw/popular_lots.csv', index=False)